# Deploy the trained dummy classifier to Azure
# Steps
1. Create a config json file to specify the subscription id, resource group, and workspace name
2. Register model to Azure machine learning
3. Create a Conda environment for scikit-learn model
4. Create inference config file
5. Create Azure container instance deployment confguration
6. Deploy the model

In [1]:
!pip install azureml-sdk

In [2]:
import os
import json
import requests

from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

In [3]:
# laoding subscription id, resource group, and work space names from config file
config_file_path = "config.json"

with open(config_file_path, 'r') as file:
    data = json.load(file)

subscription_id = data["subscription_id"]
resource_group = data["resource_group"]
workspace_name = data["workspace_name"]
region = data["region"]

In [4]:
print(resource_group)
print(workspace_name)
print(region)

dummy-classification-rg
dummy-model-classification-testws
westeurope


In [5]:
# Create workspace
ws = Workspace.create(name=workspace_name,
                      subscription_id=subscription_id,
                      resource_group=resource_group,
                      location=region)

Performing interactive authentication. Please follow the instructions on the terminal.


Interactive authentication successfully completed.


Deploying StorageAccount with name dummymodstorage6ed858f66.
Deploying AppInsights with name dummymodinsights5df00d2d.
Deployed AppInsights with name dummymodinsights5df00d2d. Took 4.55 seconds.
Deploying KeyVault with name dummymodkeyvault5a48f952.
Deployed KeyVault with name dummymodkeyvault5a48f952. Took 19.59 seconds.
Deploying Workspace with name dummy-model-classification-testws.
Deployed StorageAccount with name dummymodstorage6ed858f66. Took 24.15 seconds.
Deployed Workspace with name dummy-model-classification-testws. Took 22.09 seconds.


In [6]:
model_path = 'dummy_classifier.pkl'

In [7]:
model_name='dummy_classification_model'

In [8]:
# Register the model in Azure Machine Learning
registered_model = Model.register(model_path=model_path, model_name=model_name, workspace=ws)

Registering model dummy_classification_model


In [9]:
# Create a Conda environment for scikit-learn model
conda_env = Environment('my-conda-env')
conda_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

In [10]:
# Create an InferenceConfig
inference_config = InferenceConfig(entry_script='score.py', environment=conda_env)

In [11]:
# Specify deployment configuration for ACI
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [12]:
# Deployment
service = Model.deploy(workspace=ws,
                       name='text-classification-service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

<ipython-input-12-fae74a9b8007>:2: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-03-03 17:43:09+00:00 Creating Container Registry if not exists..
2024-03-03 17:53:10+00:00 Registering the environment..
2024-03-03 17:53:11+00:00 Building image..
2024-03-03 18:03:05+00:00 Generating deployment configuration.
2024-03-03 18:03:06+00:00 Submitting deployment to compute..
2024-03-03 18:03:11+00:00 Checking the status of deployment text-classification-service..
2024-03-03 18:04:42+00:00 Checking the status of inference endpoint text-classification-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [13]:
# Logging the deployment process
service = Webservice(ws, 'text-classification-service')
print(service.get_logs())

/bin/bash: /azureml-envs/azureml_c0ce3e6b048e85d1440ae5b5f8420e29/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c0ce3e6b048e85d1440ae5b5f8420e29/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c0ce3e6b048e85d1440ae5b5f8420e29/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2024-03-03T18:04:31,477634100+00:00 - gunicorn/run 
2024-03-03T18:04:31,473975500+00:00 - rsyslog/run 
2024-03-03T18:04:31,487254500+00:00 | gunicorn/run | 
2024-03-03T18:04:31,500529100+00:00 | gunicorn/run | ###############################################
bash: /azureml-envs/azureml_c0ce3e6b048e85d1440ae5b5f8420e29/lib/libtinfo.so.6: no version information available (required by bash)
2024-03-03T18:04:31,503321200+00:00 - nginx/run 
2024-03-03T18:04:31,508750900+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-03-03T18:04:31,511399800+00:00 | gunico

In [14]:
scoring_uri = service.scoring_uri

In [15]:
scoring_uri

'http://f254a67b-a017-49ac-a1b0-ff56b8f6487a.westeurope.azurecontainer.io/score'

In [16]:
input_data = [ "Hold on for dear life."]

# Convert the input data to a JSON string
input_data_json = json.dumps({"data": [input_data]})

In [17]:
# Define the content type
headers = {"Content-Type": "application/json"}

In [18]:
# Send an HTTP POST request to the endpoint
response = requests.post(scoring_uri, data=input_data_json, headers=headers)

In [19]:
# Check the response status code
if response.status_code == 200:
    # Parse the JSON response
    result = json.loads(response.json())
    print(result)
    # Extract the prediction (result) from the response
    prediction = result["result"][0]
    print(f"Prediction: {prediction}")
else:
    print(f"Error: {response.text}")

{'result': ['POLITICS', '0.25']}
Prediction: POLITICS


# **Delete the webservice and workspace**

In [20]:
service_name = "text-classification-service"

service = ws.webservices[service_name]
service.delete()

Running
2024-03-03 18:10:08+00:00 Check and wait for operation (a0816d13-ebd8-4707-87f2-0ea39f86773b) to finish.
2024-03-03 18:10:10+00:00 Deleting service entity.
Succeeded


In [21]:
# Specify the name of the registered model to delete
model_name = 'dummy_classification_model'

# Delete the model
registered_model = ws.models[model_name]
registered_model.delete()

In [22]:
# Delete the workspace
ws.delete(delete_dependent_resources=True)